In [30]:
import json
import pandas as pd

In [39]:
df = pd.read_csv('data/rag_filtered_150_papers_with_extraction.csv')
df

,Unnamed: 0.1,Unnamed: 0,id,full_text,filtered_text,json_output
0,0,0,115,Paper #: 115\nLiquid medium annealing for fabr...,Liquid medium annealing for fabricating durabl...,"{'perovskite_composition': None, 'electron_tra..."
1,1,1,112,Paper #: 112\nIodine reduction for reproducibl...,Iodine reduction for reproducible and high-per...,"{'perovskite_composition': 'MA0.7FA0.3PbI3', '..."
2,2,2,72,Paper #: 72\nSuppressed Ion Migration in Reduc...,T oday's perovskite solar cells (PSCs) require...,{'perovskite_composition': 'Cs0.05FA0.8MA0.15P...
3,3,3,75,Paper #: 75\nDynamical Evolution of the 2D/3D ...,UV-VIS Absorption The absorption spectra of t...,"{'perovskite_composition': '2D/3D perovskite',..."
4,4,4,40,Paper #: 40\nDamp heat-stable perovskite solar...,Damp heat-stable perovskite solar cells with t...,NaN
...,...,...,...,...,...,...
144,144,144,92,Paper #: 92\n\nBanners \n Banners New for 2018...,Electronic Table of Contents (eToC) \n Standa...,"{'perovskite_composition': None, 'electron_tra..."
145,145,145,123,Paper #: 123\nConstructing heterojunctions by ...,P erovskite solar cells (PSCs) have reached po...,{'perovskite_composition': '(FAPbI3)0.95(MAPbB...
146,146,146,31,Paper #: 31\n\nWe performed 1 H-NMR to support...,We performed 1 H-NMR to support our proposed r...,{'perovskite_composition': 'FA0.6MA0.4Sn0.6Pb0...
147,147,147,36,Paper #: 36\nUnderstanding Performance Limitin...,Understanding Performance Limiting Interfacial...,{'perovskite_composition': 'Cs0.05FA0.83MA0.17...


In [40]:
df = df[df["json_output"].isna() == False] 

In [41]:
df

,Unnamed: 0.1,Unnamed: 0,id,full_text,filtered_text,json_output
0,0,0,115,Paper #: 115\nLiquid medium annealing for fabr...,Liquid medium annealing for fabricating durabl...,"{'perovskite_composition': None, 'electron_tra..."
1,1,1,112,Paper #: 112\nIodine reduction for reproducibl...,Iodine reduction for reproducible and high-per...,"{'perovskite_composition': 'MA0.7FA0.3PbI3', '..."
2,2,2,72,Paper #: 72\nSuppressed Ion Migration in Reduc...,T oday's perovskite solar cells (PSCs) require...,{'perovskite_composition': 'Cs0.05FA0.8MA0.15P...
3,3,3,75,Paper #: 75\nDynamical Evolution of the 2D/3D ...,UV-VIS Absorption The absorption spectra of t...,"{'perovskite_composition': '2D/3D perovskite',..."
5,5,5,91,Paper #: 91\nMethylammonium Chloride Induces I...,Electronic Table of Contents (eToC) \n Email ...,"{'perovskite_composition': 'FAPbI3', 'electron..."
...,...,...,...,...,...,...
144,144,144,92,Paper #: 92\n\nBanners \n Banners New for 2018...,Electronic Table of Contents (eToC) \n Standa...,"{'perovskite_composition': None, 'electron_tra..."
145,145,145,123,Paper #: 123\nConstructing heterojunctions by ...,P erovskite solar cells (PSCs) have reached po...,{'perovskite_composition': '(FAPbI3)0.95(MAPbB...
146,146,146,31,Paper #: 31\n\nWe performed 1 H-NMR to support...,We performed 1 H-NMR to support our proposed r...,{'perovskite_composition': 'FA0.6MA0.4Sn0.6Pb0...
147,147,147,36,Paper #: 36\nUnderstanding Performance Limitin...,Understanding Performance Limiting Interfacial...,{'perovskite_composition': 'Cs0.05FA0.83MA0.17...


In [50]:
with open('data/deepseek_8bit_finetuned.json', 'r') as f:
    data = json.load(f)

In [ ]:
output = {}
i = 0
for index, row in df.iterrows():
    i += 1
    output[str(row["id"])] = row["json_output"]

In [43]:
with open('data/deepseek_8bit_finetuned_tests_nested.json', 'w') as f:
    json.dump(output, f)

In [44]:
df = pd.read_json('data/deepseek_8bit_finetuned.json')
df

ValueError: If using all scalar values, you must pass an index